In [1]:
import os
os.chdir('../')
%pwd

'/home/ehsan/PycharmProjects/Chicken-Disease-Classification/Chicken-Disease-Classification'

In [2]:
import tensorflow as tf

2024-02-22 13:41:56.595850: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

2024-02-22 21:10:23.759378: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    model_path: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_dirs, save_json

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_file=CONFIG_FILE_PATH,
            params_file=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file)
        self.params = read_yaml(params_file)
        create_dirs([self.config.artifacts_root])
        
    def get_evaluation_config(self):
        eval_config = EvaluationConfig(
            model_path=Path(self.config.training.trained_model_path),
            training_data=Path('artifacts/data_ingestion/Chicken-fecal-images'),
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config
    

In [8]:
from urllib.parse import urlparse

In [13]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.model = None
        self.score = None
        self.config = config
        
       
    def _valid_generator(self):
        data_generator_kwargs = dict(
            rescale=1.0/255,
            validation_split=0.3
        ) 
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation='bilinear'
        )
        val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator_kwargs
        )
        self.val_datagen = val_datagen.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(path: Path) ->tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.model_path)
        self._valid_generator()
        self.score = self.model.evaluate(self.val_datagen)
        
    def save_score(self):
        scores = {'loss': self.score[0], 'accuracy': self.score[1]}
        save_json(path_to_json=Path('scores.json'), data=scores)
        

In [14]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluator = Evaluation(eval_config)
    evaluator.evaluation()
    evaluator.save_score()
except Exception as e:
    raise Exception(e)

2024-02-22 21:34:52,434 - INFO - common - Loaded yaml file config/config.yaml
2024-02-22 21:34:52,437 - INFO - common - Loaded yaml file params.yaml
2024-02-22 21:34:52,438 - INFO - common - created directory artifacts
Found 116 images belonging to 2 classes.


2024-02-22 21:34:53.291779: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


8/8 [==============================] - 29s 3s/step - loss: 1.7911 - accuracy: 0.8276
2024-02-22 21:35:22,043 - INFO - common - saved json file scores.json
